In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications import VGG16, Xception, InceptionV3, ResNet50
from keras.models import Model
from keras.layers import AveragePooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
import h5py

In [3]:
train_gen = ImageDataGenerator(rescale=1./255)
validation_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

In [4]:
train_data = train_gen.flow_from_directory('../data/train', 
                                           target_size=(224, 224), 
                                           classes=['cat_train', 'dog_train'], 
                                           class_mode='binary', shuffle=False)
validation_data = validation_gen.flow_from_directory('../data/validation', 
                                                     target_size=(224, 224), 
                                                     classes=['cat', 'dog'], 
                                                     class_mode='binary', shuffle=False)
test_data = test_gen.flow_from_directory('../data', 
                                        target_size=(224, 224),
                                        classes=['test'],
                                        class_mode=None, shuffle=False)

Found 24600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [5]:
vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

In [6]:
vgg_train_featuremap = vgg_model.predict_generator(train_data, steps=(train_data.n//train_data.batch_size+1))

In [7]:
vgg_validation_featuremap = vgg_model.predict_generator(validation_data, 
                                                        steps=(validation_data.n//validation_data.batch_size+1))

In [11]:
with h5py.File('../data/vgg.h5', 'w') as f:
    f.create_dataset('train', data=vgg_train_featuremap)
    f.create_dataset('validation', data=vgg_validation_featuremap)

In [8]:
from keras.models import Sequential

In [9]:
mymodel = Sequential()
mymodel.add(Flatten(input_shape=vgg_train_featuremap.shape[1:]))
mymodel.add(Dense(256, activation='relu'))
mymodel.add(Dropout(0.5))
mymodel.add(Dense(1, activation='sigmoid'))

In [10]:
mymodel.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
import numpy as np
train_label = np.array([0]*12300+[1]*12300)

In [12]:
validation_label = np.array([0]*200 + [1]*200)

In [14]:
mymodel.fit(vgg_train_featuremap, train_label, epochs=20, batch_size=50, 
           validation_data=[vgg_validation_featuremap, validation_label])

Train on 24600 samples, validate on 400 samples
Epoch 1/20
24600/24600 [==============================] - 5s - loss: 0.1421 - acc: 0.9425 - val_loss: 0.1653 - val_acc: 0.9150

In [55]:
model_weight = mymodel.get_weights()

In [63]:
input_ten = Input(vgg_train_featuremap.shape[1:])
x = Flatten()(input_ten)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
top_model = Model(input_ten, x)

In [65]:
top_model.set_weights(model_weight)

In [107]:
input_tensor = Input(shape=(224, 224, 3))
x = vgg_model.get_layer(index=1)(input_tensor)
for i in range(2, len(vgg_model.layers)):
    x = vgg_model.get_layer(index=i)(x)
x = top_model(x)
new_model = Model(input_tensor, x)

In [110]:
for layer in new_model.layers[:-5]:
    layer.trainable = False

In [114]:
new_model.compile(optimizer=keras.optimizers.SGD(lr=1e-4, momentum=0.9), loss='binary_crossentropy', 
                  metrics=['accuracy'])

In [ ]:
new_model.fit_generator(train_data, steps_per_epoch=(train_data.n//train_data.batch_size+1), 
                       epochs=20, validation_data=validation_data, 
                       validation_steps=validation_data.n // validation_data.batch_size)

Epoch 1/20
769/769 [==============================] - 412s - loss: 0.2163 - acc: 0.9852 - val_loss: 8.6576 - val_acc: 0.4565

In [122]:
inceptionv3_model = InceptionV3(include_top=False, input_shape=(224, 224, 3))

In [51]:
incep_train_featuremap = inceptionv3_model.predict_generator(train_data, 
                                                             steps=(train_data.n//train_data.batch_size+1))

In [123]:
run_step = 
incep_validation_featuremap = inceptionv3_model.predict_generator(validation_data, 
                                                                  steps=(validation_data.n//validation_data.batch_size+1))

In [52]:
resnet_model = ResNet50(include_top=False, input_shape=(224, 224, 3))

In [53]:
res_train_featuremap = resnet_model.predict_generator(train_data, 
                                                      steps=(train_data.n//train_data.batch_size+1))

In [125]:
res_validaiont_featuremap = resnet_model.predict_generator(validation_data, 
                                                           steps=(validation_data.n//validation_data.batch_size+1))

In [121]:
import h5py

In [124]:
with h5py.File('../data/inception', 'w') as f:
    f.create_dataset('train', data=incep_train_featuremap)
    f.create_dataset('validation', data=incep_validation_featuremap)

In [126]:
with h5py.File('../data/resnet', 'w') as f:
    f.create_dataset('train', data=res_train_featuremap)
    f.create_dataset('validation', data=res_validaiont_featuremap)